In [131]:
import pandas as pd
import numpy as np
import os
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, f_regression 
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.pipeline import make_pipeline

In [165]:
def new_zillow_db():
    '''The function uses the get_connection function to connect to a database and retrieve the zillow dataset'''
    
    zillow = pd.read_sql('''SELECT p.bathroomcnt, p.bedroomcnt, p.calculatedfinishedsquarefeet,
    p.fips, p.garagecarcnt, p.lotsizesquarefeet, p.regionidcity, p.regionidcounty, p.regionidzip,
    p.roomcnt, p.yearbuilt, p.taxvaluedollarcnt, pd.transactiondate, pd.logerror
    FROM properties_2017 as p

    JOIN predictions_2017 as pd
    on p.id = pd.id
    LEFT JOIN propertylandusetype USING(propertylandusetypeid)

    WHERE propertylandusedesc IN ("Single Family Residential")
    AND pd.transactiondate BETWEEN '2017-01-01' AND '2017-12-31'
    AND p.bedroomcnt > 0
    AND p.bathroomcnt > 0
    AND p.calculatedfinishedsquarefeet IS NOT NULL
    AND p.lotsizesquarefeet IS NOT NULL
    AND p.taxvaluedollarcnt IS NOT NULL
    AND p.yearbuilt IS NOT NULL
    ;''', wrangle.get_connection('zillow'))
    return zillow


In [133]:
df = new_zillow_db()

In [134]:
df

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,lotsizesquarefeet,regionidcity,regionidcounty,regionidzip,yearbuilt,taxvaluedollarcnt,transactiondate,logerror
0,0.0,0.0,NaN,6037.0,4083.0,37688.0,3101.0,96337.0,NaN,27516.0,2017-01-01,0.055619
1,0.0,0.0,NaN,6037.0,11975.0,40227.0,3101.0,97329.0,NaN,10.0,2017-01-02,0.008669
2,0.0,0.0,NaN,6037.0,9403.0,40227.0,3101.0,97330.0,NaN,10.0,2017-01-02,-0.021896
3,0.0,0.0,NaN,6037.0,3817.0,40227.0,3101.0,97330.0,NaN,2108.0,2017-01-02,-0.017167
4,2.0,4.0,3633.0,6037.0,9826.0,40227.0,3101.0,97329.0,2005.0,296425.0,2017-01-02,0.042463
...,...,...,...,...,...,...,...,...,...,...,...,...
56074,2.0,2.0,1362.0,6037.0,6854.0,40227.0,3101.0,97330.0,1996.0,127276.0,2017-09-19,0.001082
56075,2.0,3.0,1600.0,6037.0,6709.0,40227.0,3101.0,97330.0,1992.0,180715.0,2017-09-20,-0.002245
56076,3.0,4.0,1921.0,6037.0,6763.0,40227.0,3101.0,97330.0,1990.0,162019.0,2017-09-20,0.020615
56077,3.0,4.0,1780.0,6037.0,6822.0,40227.0,3101.0,97330.0,1989.0,125923.0,2017-09-21,0.013209


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56079 entries, 0 to 56078
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   56079 non-null  float64
 1   bedroomcnt                    56079 non-null  float64
 2   calculatedfinishedsquarefeet  55848 non-null  float64
 3   fips                          56079 non-null  float64
 4   lotsizesquarefeet             55729 non-null  float64
 5   regionidcity                  55025 non-null  float64
 6   regionidcounty                56079 non-null  float64
 7   regionidzip                   55979 non-null  float64
 8   yearbuilt                     55830 non-null  float64
 9   taxvaluedollarcnt             56071 non-null  float64
 10  transactiondate               56079 non-null  object 
 11  logerror                      56079 non-null  float64
dtypes: float64(11), object(1)
memory usage: 5.1+ MB


In [141]:
df = df.dropna()

In [142]:
df.bedroomcnt = df.bedroomcnt.astype(int)
    # change year built to an integer
df.yearbuilt = df.yearbuilt.astype(int)
    # change fips to an integer
df.fips = df.fips.astype(int)
    # rename columns for readability
df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'sqft', 
                        'taxvaluedollarcnt': 'tax_value', 'yearbuilt': 'year'})

/var/folders/5g/xrhpsnj56_v3scw9s4trbgxc0000gn/T/ipykernel_3896/26307220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.bedroomcnt = df.bedroomcnt.astype(int)
/var/folders/5g/xrhpsnj56_v3scw9s4trbgxc0000gn/T/ipykernel_3896/26307220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.yearbuilt = df.yearbuilt.astype(int)
/var/folders/5g/xrhpsnj56_v3scw9s4trbgxc0000gn/T/ipykernel_3896/26307220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [143]:
df = df.rename(columns={'lotsizesquarefeet':'lot_size', 'regionidzip':'zipcode','regionidcounty':'county',
                        'regionidcity':'city'})

In [144]:
df

,bathrooms,bedrooms,sqft,fips,lot_size,city,county,zipcode,year,tax_value,transactiondate,logerror
4,2.0,4,3633.0,6037,9826.0,40227.0,3101.0,97329.0,2005,296425.0,2017-01-02,0.042463
7,2.0,3,2077.0,6037,6490.0,26964.0,3101.0,96152.0,1926,646760.0,2017-01-02,-0.040807
11,0.0,0,1200.0,6037,4054.0,12447.0,3101.0,96020.0,1972,5328.0,2017-01-02,0.069858
14,0.0,0,171.0,6037,13002.0,46298.0,3101.0,96246.0,1973,6920.0,2017-01-02,0.020950
15,0.0,0,203.0,6037,17359.0,46298.0,3101.0,96236.0,1960,14166.0,2017-01-02,0.033233
...,...,...,...,...,...,...,...,...,...,...,...,...
56074,2.0,2,1362.0,6037,6854.0,40227.0,3101.0,97330.0,1996,127276.0,2017-09-19,0.001082
56075,2.0,3,1600.0,6037,6709.0,40227.0,3101.0,97330.0,1992,180715.0,2017-09-20,-0.002245
56076,3.0,4,1921.0,6037,6763.0,40227.0,3101.0,97330.0,1990,162019.0,2017-09-20,0.020615
56077,3.0,4,1780.0,6037,6822.0,40227.0,3101.0,97330.0,1989,125923.0,2017-09-21,0.013209


In [145]:
df.describe()

,bathrooms,bedrooms,sqft,fips,lot_size,city,county,zipcode,year,tax_value,logerror
count,54553.000000,54553.000000,54553.000000,54553.000000,5.455300e+04,54553.000000,54553.000000,54553.000000,54553.000000,5.455300e+04,54553.000000
mean,2.237686,3.298517,1857.693784,6048.856177,1.123118e+04,35502.566018,2560.631918,96594.523436,1960.796253,4.525896e+05,0.016783
std,0.989100,0.931808,955.170666,21.280432,9.141415e+04,54204.959656,786.841412,4323.250953,21.698743,5.892449e+05,0.172213
min,0.000000,0.000000,20.000000,6037.000000,1.690000e+02,3491.000000,1286.000000,95982.000000,1862.000000,9.000000e+00,-4.655420
25%,2.000000,3.000000,1260.000000,6037.000000,5.686000e+03,12447.000000,2061.000000,96193.000000,1949.000000,1.866290e+05,-0.024437
50%,2.000000,3.000000,1624.000000,6037.000000,6.790000e+03,24832.000000,3101.000000,96389.000000,1958.000000,3.271200e+05,0.006741
75%,3.000000,4.000000,2200.000000,6059.000000,8.600000e+03,44116.000000,3101.000000,96987.000000,1975.000000,5.288470e+05,0.039486
max,20.000000,25.000000,26345.000000,6111.000000,6.971010e+06,396556.000000,3101.000000,399675.000000,2016.000000,3.016684e+07,5.262999


In [146]:
df = df[df['bedrooms'] < 6]
#     # remove rows with 6 or more bathrooms
df = df[df['bathrooms'] < 5]
#     # remove rows with values less than or equal to 700 square feet
df = df[df.sqft > 700]
#     # remove rows with values greater than or equal to 10_000 square feet
# df = df[df.sqft < 3000]
# remove rows with tax values greater than or equal to 600000
df = df[df.tax_value < 700000]
# remove rows with tax values less than or equal to 1000
df = df[df.tax_value > 100000]
#     # remove rows with a year less than or equal to 1899
# df = df[df.year > 1899]
df = df[df.bathrooms > 0]
df = df[df.bedrooms > 0]

In [147]:
df = df[df.lot_size < 10000]
df = df[df.lot_size > 1000]

In [157]:
# def get_zillow_data():
#     ''' This function reads in telco data from Codeup database, writes data to
#     a csv file if a local file does not exist, and returns a df.'''
#     if os.path.isfile('zillow_project.csv'):
#         # If csv file exists read in data from csv file.
#         df = pd.read_csv('zillow_project.csv', index_col=0)     
#     else:   
#         # Read fresh data from db into a DataFrame
#         df = new_zillow_db()
#         # Cache data
#         df.to_csv('zillow_project.csv')

In [166]:
def wrangle_zillow():
    '''This function acquires the zillow dataset from the Codeup database using a SQL query and returns a cleaned
    dataframe from a csv file. Observations with null values are dropped and column names are changed for
    readability. Values expected as integers are converted to integer types (year, bedrooms, fips).'''
    if os.path.isfile('zillow_project.csv'):
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow_project.csv', index_col=0)     
    else:   
        # Read fresh data from db into a DataFrame
        df = new_zillow_db()
        # Cache data
        df.to_csv('zillow_project.csv')
    # convert null values of garage count to mean value
    df.garagecarcnt = df.garagecarcnt.fillna(2.0)
    # change bedroom count to an integer
    df.bedroomcnt = df.bedroomcnt.astype(int)
    # change year built to an integer
    df.yearbuilt = df.yearbuilt.astype(int)
    # change fips to an integer
    df.fips = df.fips.astype(int)
    # change regionidcounty to an integer
    df.regionidcounty = df.regionidcounty.astype(int)
    # chang regionidcity to an integer
    df.regionidcity = df.regionidcity.astype(int)
    # rename columns for readability
    df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'sqft', 
                        'taxvaluedollarcnt': 'tax_value', 'yearbuilt': 'year', 'taxamount': 'tax_amount','lotsizesquarefeet':'lot_size', 
                        'regionidzip':'zipcode','regionidcounty':'county','regionidcity':'city','garagecarcnt':'garages'})
    # remove rows with 6 or more bedrooms
    df = df[df['bedrooms'] < 6]
    # remove rows with 5 or more bathrooms
    df = df[df['bathrooms'] < 5]
    # remove rows with values less than or equal to 700 square feet
    df = df[df.sqft > 700]
    # remove rows with values greater than or equal to 10_000 square feet
    df = df[df.sqft < 10000]
    # remove rows with tax values greater than or equal to 600000
    df = df[df.tax_value < 600000]
    # remove rows with tax values less than or equal to 1000
    df = df[df.tax_value > 100000]
    # remove rows with a year less than or equal to 1899
    df = df[df.year > 1899]
    # remove rows with lot size less than 10000 square feet
    df = df[df.lot_size < 10000]
    # remove rows with lot size greater than 1000 square feet 
    df = df[df.lot_size > 1000]

    return df

In [159]:
def split_data(df):
    '''This function takes in a dataframe and returns three dataframes, a training dataframe with 60 percent of the data, 
        a validate dataframe with 20 percent of the data and test dataframe with 20 percent of the data.'''
    # split the dataset into two, with 80 percent of the observations in train and 20 percent in test
    train, test = train_test_split(df, test_size=.2, random_state=217)
    # split the train again into two sets, using a 75/25 percent split
    train, validate = train_test_split(train, test_size=.25, random_state=217)
    # return three datasets, train (60 percent of total), validate (20 percent of total), and test (20 percent of total)
    return train, validate, test

In [160]:
def quantile_scaler_norm(a,b,c):
    '''This function applies the .QuantileTransformer method from sklearn to three arguments, a, b, and c,
    (X_train, X_validate, and X_test) and returns the scaled versions of each variable.'''
    # make the scaler
    scaler = QuantileTransformer(output_distribution='normal')
    # fit and transform the X_train variable
    X_train_quantile = pd.DataFrame(scaler.fit_transform(a))
    # transform the X_validate variable
    X_validate_quantile = pd.DataFrame(scaler.transform(b))
    # transform the X_test variable
    X_test_quantile = pd.DataFrame(scaler.transform(c))
    # return three variables, one for each newly scaled variable
    return X_train_quantile, X_validate_quantile, X_test_quantile

In [161]:
def quantile_scaler(a,b,c):
    '''This function applies the .QuantileTransformer method from sklearn to three arguments, a, b, and c,
    (X_train, X_validate, and X_test) and returns the scaled versions of each variable.'''
    # make the scaler
    scaler = QuantileTransformer()
    # fit and transform the X_train variable
    X_train_quantile = pd.DataFrame(scaler.fit_transform(a))
    # transform the X_validate variable
    X_validate_quantile = pd.DataFrame(scaler.transform(b))
    # transform the X_test variable
    X_test_quantile = pd.DataFrame(scaler.transform(c))
    # return three variables, one for each newly scaled variable
    return X_train_quantile, X_validate_quantile, X_test_quantile

In [162]:
def standard_scaler(a,b,c):
    '''This function applies the .StandardScaler method from sklearn to three arguments, a, b, and c, 
    and returns the scaled versions of each variable.'''
    # make the scaler
    scaler = StandardScaler()
    # fit and transform the X_train data
    X_train_standard = pd.DataFrame(scaler.fit_transform(a))
    # transform the X_validate data
    X_validate_standard = pd.DataFrame(scaler.transform(b))
    # transform the X_test data
    X_test_standard = pd.DataFrame(scaler.transform(c))
    # return the scaled data for each renamed variable
    return X_train_standard, X_validate_standard, X_test_standard

In [163]:
def minmax_scaler(a,b,c):
    '''This function applies the .MinMaxScaler method from sklearn to three arguments, a, b, and c,
    and returns the scaled versions of each variable.'''
    # make the scaler
    scaler = MinMaxScaler()
    # fit and transform the X_train data
    X_train_scaled = pd.DataFrame(scaler.fit_transform(a))
    # transform the X_validate data
    X_validate_scaled = pd.DataFrame(scaler.transform(b))
    # transform the X_test data
    X_test_scaled = pd.DataFrame(scaler.transform(c))
    # return the scaled data for each renamed variable
    return X_train_scaled, X_validate_scaled, X_test_scaled

In [164]:
def robust_scaler(a,b,c):
    '''This function applies the .RobustScaler method from sklearn to three arguments, a, b, and c,
    and returns the scaled versions of each variable.'''
    # make the scaler
    scaler = RobustScaler()
    # fit and transform the X_train data
    X_train_robust = pd.DataFrame(scaler.fit_transform(a))
    # transform the X_validate data
    X_validate_robust = pd.DataFrame(scaler.transform(b))
    # transform the X_test data
    X_test_robust = pd.DataFrame(scaler.transform(c))
    # return the scaled data for each renamed variable
    return X_train_robust, X_validate_robust, X_test_robust